In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from itertools import product

The extensive experiments using different
classifiers and combinations of different vision and text features
on multiple sentiment scenarios showed that late fusion is more
effective than early fusion. The analysis of explainability revealed
that in late fusion, the classes were predominantly influenced by
the respective uni-modal prediction probabilities, indicating the necessity for extracting more appropriate features through additional
fine-tuning procedures

In [2]:
fusion_df = pd.read_csv("outputs_perModel.csv", 
                         encoding='utf-8', 
                         header=0)

temp = pd.read_csv('earlyFusion_results_df.csv')

fusion_df = fusion_df.merge(
    temp, on=["video_id", "clip_id", "annotation_label", "mode"], how="inner"
)

In [3]:
fusion_df

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,...,trimodal_preds,text_audio_prob_0,text_audio_prob_1,text_audio_preds,audio_video_prob_0,audio_video_prob_1,audio_video_preds,text_video_prob_0,text_video_prob_1,text_video_preds
0,03bSnISJMiM,11,a lot of sad part,train,0.0,0.997359,0.002641,0,0.399598,0.600402,...,1,0.545542,0.454458,0,0.410912,0.589088,1,0.547833,0.452167,0
1,03bSnISJMiM,10,there is sad part,train,0.0,0.997300,0.002700,0,0.439418,0.560582,...,0,0.546001,0.453999,0,0.439674,0.560326,1,0.602740,0.397260,0
2,03bSnISJMiM,13,and it a really funny,train,1.0,0.002918,0.997082,1,0.350468,0.649532,...,1,0.354512,0.645488,1,0.370054,0.629946,1,0.382594,0.617406,1
3,03bSnISJMiM,12,but it wa really really awesome,train,1.0,0.002906,0.997094,1,0.347897,0.652103,...,1,0.349426,0.650574,1,0.358518,0.641482,1,0.386552,0.613448,1
4,03bSnISJMiM,1,anyhow it wa really good,train,1.0,0.002872,0.997128,1,0.435212,0.564788,...,1,0.516387,0.483613,0,0.365942,0.634058,1,0.582906,0.417094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,zhpQhgha_KU,30,because there really wa not all that much to i...,test,0.0,0.997182,0.002818,0,0.597331,0.402669,...,0,0.559316,0.440684,0,0.546081,0.453919,0,0.542318,0.457682,0
2195,zhpQhgha_KU,35,so if you like to hear a like more positive re...,test,1.0,0.002930,0.997070,1,0.399418,0.600582,...,1,0.334322,0.665678,1,0.398728,0.601272,1,0.396127,0.603873,1
2196,zhpQhgha_KU,34,and she really enjoyed the film,test,1.0,0.002905,0.997095,1,0.341855,0.658145,...,1,0.344932,0.655068,1,0.354687,0.645313,1,0.386174,0.613826,1
2197,zhpQhgha_KU,33,if you do want to see somebody who is possibly...,test,0.0,0.996939,0.003061,0,0.406756,0.593244,...,1,0.481836,0.518164,1,0.461334,0.538666,1,0.524029,0.475971,0


In [8]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping

In [9]:
fusion_df[(fusion_df['video_id'] == 'yDtzw_Y-7RU') & (fusion_df['clip_id'] == 18)]

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,audioLabel,probVideo_0,probVideo_1,videoLabel,trimodal_prob_0,trimodal_prob_1,trimodal_preds,text_audio_prob_0,text_audio_prob_1,text_audio_preds,audio_video_prob_0,audio_video_prob_1,audio_video_preds,text_video_prob_0,text_video_prob_1,text_video_preds
2140,yDtzw_Y-7RU,18,also about the animation the people just like ...,test,0.0,0.996969,0.003031,0,0.471698,0.528302,1,0.51943,0.48057,0,0.269035,0.730965,1,0.507976,0.492024,0,0.458879,0.541121,1,0.609702,0.390298,0


In [4]:
def printReport(df, column_name):
    subsets = ['train', 'valid', 'test']
        
    print("Classification Report for the Whole Dataset:")
    print(classification_report(df['annotation_label'], df[column_name], digits=4))
    
    for subset in subsets:
        subset_df = df[df['mode'] == subset]
        if not subset_df.empty:
            print(f"\nClassification Report for {subset.capitalize()} Subset:")
            print(classification_report(subset_df['annotation_label'], subset_df[column_name], digits=4))

---
# Hard Fusion (Based on labels)   
---

https://chatgpt.com/c/67981b72-fdf4-8002-95bd-5a75e258e4c3

# Majority Voting

In [7]:
# Function for majority voting
def majority_vote(row):
    # Collect the predicted labels
    labels = [row['textLabel'], 
              row['audioLabel'],
              row['videoLabel'],
              row['trimodal_preds'],
              row['text_audio_preds'],
              row['audio_video_preds'],
              row['text_video_preds'],
             ]
    
    # Count the occurrences of each class
    class_0_count = labels.count(0)
    class_1_count = labels.count(1)
    
    # Apply majority voting
    if class_0_count > class_1_count:
        return 0
    elif class_1_count > class_0_count:
        return 1
    else:
        # Tie-breaking rule (e.g., prioritize textLabel or assign -1 for a tie)
        return row['textLabel']  # Example: prioritizing text predictions

In [8]:
# Apply majority voting to each row
fusion_df['majorityLabel'] = fusion_df.apply(majority_vote, axis=1)

In [9]:
fusion_df

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,...,text_audio_prob_0,text_audio_prob_1,text_audio_preds,audio_video_prob_0,audio_video_prob_1,audio_video_preds,text_video_prob_0,text_video_prob_1,text_video_preds,majorityLabel
0,03bSnISJMiM,11,a lot of sad part,train,0.0,0.997359,0.002641,0,0.399598,0.600402,...,0.545542,0.454458,0,0.410912,0.589088,1,0.547833,0.452167,0,1
1,03bSnISJMiM,10,there is sad part,train,0.0,0.997300,0.002700,0,0.439418,0.560582,...,0.546001,0.453999,0,0.439674,0.560326,1,0.602740,0.397260,0,0
2,03bSnISJMiM,13,and it a really funny,train,1.0,0.002918,0.997082,1,0.350468,0.649532,...,0.354512,0.645488,1,0.370054,0.629946,1,0.382594,0.617406,1,1
3,03bSnISJMiM,12,but it wa really really awesome,train,1.0,0.002906,0.997094,1,0.347897,0.652103,...,0.349426,0.650574,1,0.358518,0.641482,1,0.386552,0.613448,1,1
4,03bSnISJMiM,1,anyhow it wa really good,train,1.0,0.002872,0.997128,1,0.435212,0.564788,...,0.516387,0.483613,0,0.365942,0.634058,1,0.582906,0.417094,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,zhpQhgha_KU,30,because there really wa not all that much to i...,test,0.0,0.997182,0.002818,0,0.597331,0.402669,...,0.559316,0.440684,0,0.546081,0.453919,0,0.542318,0.457682,0,0
2195,zhpQhgha_KU,35,so if you like to hear a like more positive re...,test,1.0,0.002930,0.997070,1,0.399418,0.600582,...,0.334322,0.665678,1,0.398728,0.601272,1,0.396127,0.603873,1,1
2196,zhpQhgha_KU,34,and she really enjoyed the film,test,1.0,0.002905,0.997095,1,0.341855,0.658145,...,0.344932,0.655068,1,0.354687,0.645313,1,0.386174,0.613826,1,1
2197,zhpQhgha_KU,33,if you do want to see somebody who is possibly...,test,0.0,0.996939,0.003061,0,0.406756,0.593244,...,0.481836,0.518164,1,0.461334,0.538666,1,0.524029,0.475971,0,1


In [10]:
printReport(fusion_df, 'majorityLabel')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8277    0.6432    0.7239      1023
         1.0     0.7400    0.8835    0.8054      1176

    accuracy                         0.7717      2199
   macro avg     0.7839    0.7634    0.7646      2199
weighted avg     0.7808    0.7717    0.7675      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8510    0.6830    0.7578       552
         1.0     0.7919    0.9098    0.8468       732

    accuracy                         0.8123      1284
   macro avg     0.8215    0.7964    0.8023      1284
weighted avg     0.8173    0.8123    0.8085      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.6667    0.4783    0.5570        92
         1.0     0.7055    0.8394    0.7667       137

    accuracy                         0.6943       229
 

# Weighted Voting

## Weights for each modality based on validation set label's accuracy

In [11]:
def calculate_weights(df, mode='valid'):
    """Calculate weights for each modality based on validation accuracy."""
    validation_df = df[df['mode'] == mode]
    
    text_accuracy = (validation_df['textLabel'] == validation_df['annotation_label']).mean()
    audio_accuracy = (validation_df['audioLabel'] == validation_df['annotation_label']).mean()
    video_accuracy = (validation_df['videoLabel'] == validation_df['annotation_label']).mean()
    trimodal_accuracy = (validation_df['trimodal_preds'] == validation_df['annotation_label']).mean()
    text_audio_accuracy = (validation_df['text_audio_preds'] == validation_df['annotation_label']).mean()
    audio_video_accuracy = (validation_df['audio_video_preds'] == validation_df['annotation_label']).mean()
    text_video_accuracy = (validation_df['text_video_preds'] == validation_df['annotation_label']).mean()
    
    return {
        'text': text_accuracy,
        'audio': audio_accuracy,
        'video': video_accuracy,
        'trimodal': trimodal_accuracy,
        'text_audio': text_audio_accuracy,
        'audio_video': audio_video_accuracy,
        'text_video': text_video_accuracy
    }

In [12]:
weights = calculate_weights(fusion_df)

print(weights)

{'text': 0.8646288209606987, 'audio': 0.6069868995633187, 'video': 0.45414847161572053, 'trimodal': 0.6550218340611353, 'text_audio': 0.8558951965065502, 'audio_video': 0.5720524017467249, 'text_video': 0.7161572052401747}


In [15]:
def weighted_voting_fusion(df, weights):
    """Perform late fusion using Weighted Voting."""
    def fused_label(row):
        # Calculate the weighted votes
        votes = Counter({
            row['textLabel']: weights['text'],
            row['audioLabel']: weights['audio'],
            row['videoLabel']: weights['video'],
            row['trimodal_preds']: weights['trimodal'],
            row['text_audio_preds']: weights['text_audio'],
            row['audio_video_preds']: weights['audio_video'],
            row['text_video_preds']: weights['text_video']
        })
        # Return the label with the highest weight
        return votes.most_common(1)[0][0]

    df['weightedVotingOnLabel_validationAccuracyWeights_Label'] = df.apply(fused_label, axis=1)
    return df

In [16]:
fusion_df = weighted_voting_fusion(fusion_df, weights)

In [17]:
fusion_df

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,...,text_audio_prob_1,text_audio_preds,audio_video_prob_0,audio_video_prob_1,audio_video_preds,text_video_prob_0,text_video_prob_1,text_video_preds,majorityLabel,weightedVotingOnLabel_validationAccuracyWeights_Label
0,03bSnISJMiM,11,a lot of sad part,train,0.0,0.997359,0.002641,0,0.399598,0.600402,...,0.454458,0,0.410912,0.589088,1,0.547833,0.452167,0,1,0
1,03bSnISJMiM,10,there is sad part,train,0.0,0.997300,0.002700,0,0.439418,0.560582,...,0.453999,0,0.439674,0.560326,1,0.602740,0.397260,0,0,0
2,03bSnISJMiM,13,and it a really funny,train,1.0,0.002918,0.997082,1,0.350468,0.649532,...,0.645488,1,0.370054,0.629946,1,0.382594,0.617406,1,1,1
3,03bSnISJMiM,12,but it wa really really awesome,train,1.0,0.002906,0.997094,1,0.347897,0.652103,...,0.650574,1,0.358518,0.641482,1,0.386552,0.613448,1,1,1
4,03bSnISJMiM,1,anyhow it wa really good,train,1.0,0.002872,0.997128,1,0.435212,0.564788,...,0.483613,0,0.365942,0.634058,1,0.582906,0.417094,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,zhpQhgha_KU,30,because there really wa not all that much to i...,test,0.0,0.997182,0.002818,0,0.597331,0.402669,...,0.440684,0,0.546081,0.453919,0,0.542318,0.457682,0,0,0
2195,zhpQhgha_KU,35,so if you like to hear a like more positive re...,test,1.0,0.002930,0.997070,1,0.399418,0.600582,...,0.665678,1,0.398728,0.601272,1,0.396127,0.603873,1,1,1
2196,zhpQhgha_KU,34,and she really enjoyed the film,test,1.0,0.002905,0.997095,1,0.341855,0.658145,...,0.655068,1,0.354687,0.645313,1,0.386174,0.613826,1,1,1
2197,zhpQhgha_KU,33,if you do want to see somebody who is possibly...,test,0.0,0.996939,0.003061,0,0.406756,0.593244,...,0.518164,1,0.461334,0.538666,1,0.524029,0.475971,0,1,0


In [18]:
printReport(fusion_df, 'weightedVotingOnLabel_validationAccuracyWeights_Label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.5837    0.7087    0.6402      1023
         1.0     0.6886    0.5604    0.6179      1176

    accuracy                         0.6294      2199
   macro avg     0.6362    0.6345    0.6290      2199
weighted avg     0.6398    0.6294    0.6283      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.4464    0.5580    0.4960       552
         1.0     0.5892    0.4781    0.5279       732

    accuracy                         0.5125      1284
   macro avg     0.5178    0.5181    0.5119      1284
weighted avg     0.5278    0.5125    0.5142      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.6103    0.9022    0.7281        92
         1.0     0.9032    0.6131    0.7304       137

    accuracy                         0.7293       229
 

## Weights for each modality using logistic regression on labels

There is no reason for us to calculate the weights using logistic regression on the predicted labels (not probabilities) since they will give the same results.

The test accuracy remains the same whether weights for each modality are calculated using logistic regression on the validation set labels or directly based on validation set accuracy because both methods fundamentally measure the reliability of each modality. 

Logistic regression, when applied to labels, essentially learns a simple model that reflects the alignment between each modality’s predictions and the ground truth, which is conceptually similar to calculating validation accuracy. 

In both cases, the derived weights reflect the relative reliability of the modalities and are normalized to ensure comparability. Consequently, the weighted voting process amplifies the influence of more reliable modalities in the same way, leading to similar test accuracy outcomes.

In [20]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score

# def calculate_weights_with_logistic_regression(df, mode='valid'):
#     """Calculate weights for each modality using logistic regression."""
#     validation_df = df[df['mode'] == mode]
#     X = validation_df[['probText_0', 'probText_1', 'probAudio_0', 'probAudio_1', 'probVideo_0', 'probVideo_1']]
#     y = validation_df['annotation_label']
    
#     # Encode labels as integers
#     le = LabelEncoder()
#     y_encoded = le.fit_transform(y)

#     # Train logistic regression models for each modality
#     models = {}
#     accuracies = {}
    
#     for modality, cols in {
#         'text': ['probText_0', 'probText_1'],
#         'audio': ['probAudio_0', 'probAudio_1'],
#         'video': ['probVideo_0', 'probVideo_1']
#     }.items():
#         X_modality = validation_df[cols]
#         model = LogisticRegression()
#         model.fit(X_modality, y_encoded)
#         models[modality] = model
        
#         # Evaluate accuracy on the validation subset
#         y_pred = model.predict(X_modality)
#         accuracies[modality] = accuracy_score(y_encoded, y_pred)

#     # Normalize weights so they sum to 1
#     total_accuracy = sum(accuracies.values())
#     weights = {modality: acc / total_accuracy for modality, acc in accuracies.items()}

#     return weights

# k-NN

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

def fusion_knn_rule_with_tuning_label(df, k_range, text="knnLabel_label", train_mode="valid"):
    """
    Apply k-Nearest Neighbors (kNN) for late fusion with hyperparameter tuning on k using 5-fold cross-validation.

    Parameters:
    - df: pandas DataFrame with columns probText_0, probAudio_0, probVideo_0, probText_1, probAudio_1, probVideo_1
    - k_range: list of int, range of k values to tune (e.g., [1, 3, 5, 7, 9])
    - text: str, the name of the column to store the resulting labels
    - mode_column: str, the name of the column indicating the dataset mode (e.g., 'train', 'valid')
    - mode_value: str, the value in mode_column to use for validation (e.g., 'valid')

    Returns:
    - df: DataFrame with the new column containing the fused labels
    - best_k: int, the best k value found during hyperparameter tuning
    """
    # Filter the validation subset
    valid_df = df[df['mode'] == train_mode]

    # Extract features (probabilities) and labels
    X = valid_df[['textLabel', 'audioLabel', 'videoLabel', 'trimodal_preds', 'text_audio_preds', 'audio_video_preds', 'text_video_preds']]
    y = valid_df['annotation_label']  # Assuming you have a 'label' column in your DataFrame

    # Initialize kNN classifier
    knn = KNeighborsClassifier()

    # Set up GridSearchCV for hyperparameter tuning
    param_grid = {'n_neighbors': range(1, k_range)}
    grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')  # 5-fold cross-validation

    # Perform hyperparameter tuning
    grid_search.fit(X, y)

    # Get the best k value
    best_k = grid_search.best_params_['n_neighbors']
    print("Best k: ", best_k)

    # Train the kNN model on the entire validation set using the best k
    best_knn = KNeighborsClassifier(n_neighbors=best_k)
    best_knn.fit(X, y)

    # Predict labels for the entire dataset using the best kNN model
    X_all = df[['textLabel', 'audioLabel', 'videoLabel', 'trimodal_preds', 'text_audio_preds', 'audio_video_preds', 'text_video_preds']]
    df[text] = best_knn.predict(X_all)

    return df

In [20]:
fusion_df = fusion_knn_rule_with_tuning_label(fusion_df, k_range=50)

Best k:  7


In [21]:
printReport(fusion_df, 'knnLabel_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.6623    0.6999    0.6806      1023
         1.0     0.7254    0.6896    0.7071      1176

    accuracy                         0.6944      2199
   macro avg     0.6939    0.6948    0.6938      2199
weighted avg     0.6961    0.6944    0.6948      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.5298    0.5960    0.5610       552
         1.0     0.6637    0.6011    0.6308       732

    accuracy                         0.5989      1284
   macro avg     0.5967    0.5986    0.5959      1284
weighted avg     0.6061    0.5989    0.6008      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8316    0.8587    0.8449        92
         1.0     0.9030    0.8832    0.8930       137

    accuracy                         0.8734       229
 

---
# Soft Fusion (Based on class posterior probabilities)   
---

# Averaging

Combine the predicted probabilities by taking their arithmetic mean.

In [23]:
def averaging_fusion(df, weights=[1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7], text='averaging_label'):
    # Normalize weights to ensure they sum to 1
    weights = [w / sum(weights) for w in weights]

    def avg_prob_label(row):
        # Calculate weighted probabilities for class 0 and class 1
        avg_prob_0 = (row['probText_0'] * weights[0] +
                      row['probAudio_0'] * weights[1] +
                      row['probVideo_0'] * weights[2] +
                      row['trimodal_prob_0'] * weights[3] +
                      row['text_audio_prob_0'] * weights[4] +
                      row['audio_video_prob_0'] * weights[5] +
                      row['text_video_prob_0'] * weights[6])
        avg_prob_1 = (row['probText_1'] * weights[0] +
                      row['probAudio_1'] * weights[1] +
                      row['probVideo_1'] * weights[2] +
                      row['trimodal_prob_1'] * weights[3] +
                      row['text_audio_prob_1'] * weights[4] +
                      row['audio_video_prob_1'] * weights[5] +
                      row['text_video_prob_1'] * weights[6])
        
        # Return the final label (0 or 1) based on the higher weighted probability
        return 0 if avg_prob_0 > avg_prob_1 else 1

    # Apply the nested function to each row of the DataFrame
    df[text] = df.apply(avg_prob_label, axis=1)
    return df

In [24]:
fusion_df = averaging_fusion(fusion_df)

In [25]:
printReport(fusion_df, 'averaging_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8750    0.7801    0.8248      1023
         1.0     0.8252    0.9031    0.8624      1176

    accuracy                         0.8458      2199
   macro avg     0.8501    0.8416    0.8436      2199
weighted avg     0.8484    0.8458    0.8449      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.9051    0.8297    0.8658       552
         1.0     0.8792    0.9344    0.9060       732

    accuracy                         0.8894      1284
   macro avg     0.8922    0.8821    0.8859      1284
weighted avg     0.8903    0.8894    0.8887      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.7831    0.7065    0.7429        92
         1.0     0.8151    0.8686    0.8410       137

    accuracy                         0.8035       229
 

## Weighted Averaging

### Weights calculating my maximizing the validation set accuracy

In [26]:
from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import normalize
import numpy as np

def calculate_weights_logistic(df):
    """
    Calculate weights using logistic regression based on probabilities for text, audio, and video.

    Parameters:
    - df: DataFrame containing probabilities and target labels.
    - target_column: Name of the column containing the true labels (0 or 1).

    Returns:
    - Normalized weights for text, audio, and video.
    """

    df = df[fusion_df['mode'] == 'valid']
    
    # Extract feature columns (probabilities for text, audio, video for class 1)
    X = df[['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1']].values
    y = df['annotation_label'].values  # True labels

    # Fit logistic regression model
    model = LogisticRegression()
    model.fit(X, y)
    
    # Extract coefficients and normalize to sum to 1
    raw_weights = np.abs(model.coef_[0])  # Take absolute values of coefficients
    normalized_weights = raw_weights / raw_weights.sum()

    return normalized_weights.tolist()

In [27]:
weights_weightedAvg = calculate_weights_logistic(fusion_df)

print(weights_weightedAvg)

[0.5879023514442369, 0.008858806716089572, 0.04878033413320987, 0.11868072407964249, 0.10542304896309844, 0.029106497554291545, 0.10124823710943116]


In [28]:
fusion_df = averaging_fusion(fusion_df, weights=weights_weightedAvg, text='weightedAveraging_label')

In [29]:
printReport(fusion_df, 'weightedAveraging_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8649    0.9013    0.8827      1023
         1.0     0.9109    0.8776    0.8939      1176

    accuracy                         0.8886      2199
   macro avg     0.8879    0.8894    0.8883      2199
weighted avg     0.8895    0.8886    0.8887      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8898    0.9366    0.9126       552
         1.0     0.9502    0.9126    0.9310       732

    accuracy                         0.9229      1284
   macro avg     0.9200    0.9246    0.9218      1284
weighted avg     0.9243    0.9229    0.9231      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

# Min-Max Rule

In [30]:
def fusion_minmax_rule(df, rule, text="minmax_label"):
    """
    Apply the min or max rule for late fusion to assign labels based on probabilities.

    Parameters:
    - df: pandas DataFrame with columns probText_0, probAudio_0, probVideo_0, probText_1, probAudio_1, probVideo_1
    - rule: str, "min" or "max" to specify which fusion rule to use
    - text: str, the name of the column to store the resulting labels

    Returns:
    - df: DataFrame with the new column containing the fused labels
    """
    def minmax_prob_label(row):
        # Extract probabilities for each class
        probs_0 = [row['probText_0'], row['probAudio_0'], row['probVideo_0'],row['trimodal_prob_0'],row['text_audio_prob_0'],row['audio_video_prob_0'],row['text_video_prob_0']]
        probs_1 = [row['probText_1'], row['probAudio_1'], row['probVideo_1'],row['trimodal_prob_1'],row['text_audio_prob_1'],row['audio_video_prob_1'],row['text_video_prob_1']]

        # Apply the selected rule
        if rule == "min":
            fused_prob_0 = min(probs_0)
            fused_prob_1 = min(probs_1)
        elif rule == "max":
            fused_prob_0 = max(probs_0)
            fused_prob_1 = max(probs_1)

        # Return the final label (0 or 1) based on the higher fused probability
        
        return 0 if fused_prob_0 > fused_prob_1 else 1

    # Apply the nested function to each row of the DataFrame
    df[text] = df.apply(minmax_prob_label, axis=1)
    return df

In [31]:
fusion_df = fusion_minmax_rule(fusion_df, rule="max", text="max_label")

In [32]:
fusion_df

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,...,audio_video_preds,text_video_prob_0,text_video_prob_1,text_video_preds,majorityLabel,weightedVotingOnLabel_validationAccuracyWeights_Label,knnLabel_label,averaging_label,weightedAveraging_label,max_label
0,03bSnISJMiM,11,a lot of sad part,train,0.0,0.997359,0.002641,0,0.399598,0.600402,...,1,0.547833,0.452167,0,1,0,0.0,0,0,0
1,03bSnISJMiM,10,there is sad part,train,0.0,0.997300,0.002700,0,0.439418,0.560582,...,1,0.602740,0.397260,0,0,0,0.0,0,0,0
2,03bSnISJMiM,13,and it a really funny,train,1.0,0.002918,0.997082,1,0.350468,0.649532,...,1,0.382594,0.617406,1,1,1,1.0,1,1,1
3,03bSnISJMiM,12,but it wa really really awesome,train,1.0,0.002906,0.997094,1,0.347897,0.652103,...,1,0.386552,0.613448,1,1,1,1.0,1,1,1
4,03bSnISJMiM,1,anyhow it wa really good,train,1.0,0.002872,0.997128,1,0.435212,0.564788,...,1,0.582906,0.417094,0,1,0,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,zhpQhgha_KU,30,because there really wa not all that much to i...,test,0.0,0.997182,0.002818,0,0.597331,0.402669,...,0,0.542318,0.457682,0,0,0,0.0,0,0,0
2195,zhpQhgha_KU,35,so if you like to hear a like more positive re...,test,1.0,0.002930,0.997070,1,0.399418,0.600582,...,1,0.396127,0.603873,1,1,1,1.0,1,1,1
2196,zhpQhgha_KU,34,and she really enjoyed the film,test,1.0,0.002905,0.997095,1,0.341855,0.658145,...,1,0.386174,0.613826,1,1,1,1.0,1,1,1
2197,zhpQhgha_KU,33,if you do want to see somebody who is possibly...,test,0.0,0.996939,0.003061,0,0.406756,0.593244,...,1,0.524029,0.475971,0,1,0,0.0,0,0,0


In [33]:
printReport(fusion_df, 'max_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8648    0.9003    0.8822      1023
         1.0     0.9101    0.8776    0.8935      1176

    accuracy                         0.8881      2199
   macro avg     0.8874    0.8889    0.8878      2199
weighted avg     0.8890    0.8881    0.8882      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8897    0.9348    0.9117       552
         1.0     0.9489    0.9126    0.9304       732

    accuracy                         0.9221      1284
   macro avg     0.9193    0.9237    0.9210      1284
weighted avg     0.9234    0.9221    0.9223      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

In [34]:
fusion_df = fusion_minmax_rule(fusion_df, rule="min", text="min_label")

In [35]:
printReport(fusion_df, 'min_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8648    0.9003    0.8822      1023
         1.0     0.9101    0.8776    0.8935      1176

    accuracy                         0.8881      2199
   macro avg     0.8874    0.8889    0.8878      2199
weighted avg     0.8890    0.8881    0.8882      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8897    0.9348    0.9117       552
         1.0     0.9489    0.9126    0.9304       732

    accuracy                         0.9221      1284
   macro avg     0.9193    0.9237    0.9210      1284
weighted avg     0.9234    0.9221    0.9223      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

In [36]:
fusion_df

,video_id,clip_id,processed_text,mode,annotation_label,probText_0,probText_1,textLabel,probAudio_0,probAudio_1,...,text_video_prob_0,text_video_prob_1,text_video_preds,majorityLabel,weightedVotingOnLabel_validationAccuracyWeights_Label,knnLabel_label,averaging_label,weightedAveraging_label,max_label,min_label
0,03bSnISJMiM,11,a lot of sad part,train,0.0,0.997359,0.002641,0,0.399598,0.600402,...,0.547833,0.452167,0,1,0,0.0,0,0,0,0
1,03bSnISJMiM,10,there is sad part,train,0.0,0.997300,0.002700,0,0.439418,0.560582,...,0.602740,0.397260,0,0,0,0.0,0,0,0,0
2,03bSnISJMiM,13,and it a really funny,train,1.0,0.002918,0.997082,1,0.350468,0.649532,...,0.382594,0.617406,1,1,1,1.0,1,1,1,1
3,03bSnISJMiM,12,but it wa really really awesome,train,1.0,0.002906,0.997094,1,0.347897,0.652103,...,0.386552,0.613448,1,1,1,1.0,1,1,1,1
4,03bSnISJMiM,1,anyhow it wa really good,train,1.0,0.002872,0.997128,1,0.435212,0.564788,...,0.582906,0.417094,0,1,0,0.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,zhpQhgha_KU,30,because there really wa not all that much to i...,test,0.0,0.997182,0.002818,0,0.597331,0.402669,...,0.542318,0.457682,0,0,0,0.0,0,0,0,0
2195,zhpQhgha_KU,35,so if you like to hear a like more positive re...,test,1.0,0.002930,0.997070,1,0.399418,0.600582,...,0.396127,0.603873,1,1,1,1.0,1,1,1,1
2196,zhpQhgha_KU,34,and she really enjoyed the film,test,1.0,0.002905,0.997095,1,0.341855,0.658145,...,0.386174,0.613826,1,1,1,1.0,1,1,1,1
2197,zhpQhgha_KU,33,if you do want to see somebody who is possibly...,test,0.0,0.996939,0.003061,0,0.406756,0.593244,...,0.524029,0.475971,0,1,0,0.0,0,0,0,0


Min and max rules give the same results when the probability rankings across modalities are consistent.   
-> This occurs when the relative order of the probabilities for each class (e.g., class 0 and class 1) remains the same across all modalities.   
In such cases, the minimum and maximum probabilities for each class will still correctly identify the class with the higher overall likelihood, leading to identical final labels.
SO  
-> Agreement Among Modalities  
If the probabilities from all modalities consistently favor the same class (e.g., class 0 or class 1), both the AVERAGING and MIN-MAX rules will likely produce the same final label.  

For example:   
Modalities all strongly favor class 0:   
Text: 0.8, Audio: 0.7, Video: 0.9 → Final decision = class 0.   
In such cases, the actual method of fusion (min, max, or average) does not matter because all modalities already agree.   

When there is significant disagreement among modalities, these rules can diverge, and the choice of rule impacts the final classification.


Balancing Effect of Averaging   
The averaging rule aggregates probabilities, smoothing out extremes.  
This is functionally similar to the way the min and max rules act in certain cases:  
- The min rule emphasizes the weakest agreement across modalities.
- The max rule focuses on the strongest agreement.
- Averaging lies between these extremes, blending the influence of all modalities equally. If no extreme disagreement exists among modalities, the averaged probability may still reflect the dominant class.

---

Situations That Prevent Consistent Results (That would make Min and Max rule give different results)
Differences between the min and max rules arise when the rankings of class probabilities are not consistent across modalities. This happens when:

- One or more modalities strongly favor one class while others favor another.
- Disagreement arises due to noisy or unreliable modalities.

# Product Rule

In [37]:
def product_fusion(df, text='product_label'):
    def prod_prob_label(row):
        # Calculate product of probabilities for class 0 and class 1
        prob_0 = (row['probText_0']*row['probAudio_0']*row['probVideo_0']*row['trimodal_prob_0']*row['text_audio_prob_0']*row['audio_video_prob_0']*row['text_video_prob_0'])
        prob_1 = (row['probText_1']*row['probAudio_1']*row['probVideo_1']*row['trimodal_prob_1']*row['text_audio_prob_1']*row['audio_video_prob_1']*row['text_video_prob_1'])
        
        # Return the final label (0 or 1) based on the higher product probability
        return 0 if prob_0 > prob_1 else 1

    # Apply the nested function to each row of the DataFrame
    df[text] = df.apply(prod_prob_label, axis=1)
    return df

In [38]:
fusion_df = product_fusion(fusion_df, text="product_label")

In [39]:
printReport(fusion_df, 'product_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8671    0.8993    0.8829      1023
         1.0     0.9095    0.8801    0.8946      1176

    accuracy                         0.8890      2199
   macro avg     0.8883    0.8897    0.8887      2199
weighted avg     0.8898    0.8890    0.8891      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8946    0.9384    0.9160       552
         1.0     0.9518    0.9167    0.9339       732

    accuracy                         0.9260      1284
   macro avg     0.9232    0.9275    0.9249      1284
weighted avg     0.9272    0.9260    0.9262      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

# k-NN

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

def fusion_knn_rule_with_tuning(df, k_range, text="knn_label", train_mode="valid"):
    """
    Apply k-Nearest Neighbors (kNN) for late fusion with hyperparameter tuning on k using 5-fold cross-validation.

    Parameters:
    - df: pandas DataFrame with columns probText_0, probAudio_0, probVideo_0, probText_1, probAudio_1, probVideo_1
    - k_range: list of int, range of k values to tune (e.g., [1, 3, 5, 7, 9])
    - text: str, the name of the column to store the resulting labels
    - mode_column: str, the name of the column indicating the dataset mode (e.g., 'train', 'valid')
    - mode_value: str, the value in mode_column to use for validation (e.g., 'valid')

    Returns:
    - df: DataFrame with the new column containing the fused labels
    - best_k: int, the best k value found during hyperparameter tuning
    """
    # Filter the validation subset
    valid_df = df[df['mode'] == train_mode]

    # Extract features (probabilities) and labels
    X = valid_df[['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1', 
                  'probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0', 'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0']]
    y = valid_df['annotation_label']  # Assuming you have a 'label' column in your DataFrame

    # Initialize kNN classifier
    knn = KNeighborsClassifier()

    # Set up GridSearchCV for hyperparameter tuning
    param_grid = {'n_neighbors': range(1, k_range)}
    grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')  # 5-fold cross-validation

    # Perform hyperparameter tuning
    grid_search.fit(X, y)

    # Get the best k value
    best_k = grid_search.best_params_['n_neighbors']
    print("Best k: ", best_k)

    # Train the kNN model on the entire validation set using the best k
    best_knn = KNeighborsClassifier(n_neighbors=best_k)
    best_knn.fit(X, y)

    # Predict labels for the entire dataset using the best kNN model
    X_all = df[['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1', 
                'probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0', 'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0']]
    df[text] = best_knn.predict(X_all)

    return df

In [41]:
fusion_df = fusion_knn_rule_with_tuning(fusion_df, k_range=50)

Best k:  8


In [42]:
printReport(fusion_df, 'knn_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8647    0.8935    0.8788      1023
         1.0     0.9046    0.8784    0.8913      1176

    accuracy                         0.8854      2199
   macro avg     0.8846    0.8859    0.8851      2199
weighted avg     0.8860    0.8854    0.8855      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8904    0.9275    0.9086       552
         1.0     0.9436    0.9139    0.9285       732

    accuracy                         0.9198      1284
   macro avg     0.9170    0.9207    0.9186      1284
weighted avg     0.9207    0.9198    0.9200      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8163    0.8696    0.8421        92
         1.0     0.9084    0.8686    0.8881       137

    accuracy                         0.8690       229
 

# Naive Bayes

In [43]:
from sklearn.naive_bayes import GaussianNB
def naive_bayes_fusion(df, text='naive_bayes_label'):
    # df_train = df[df['mode'].isin(['train', 'valid'])]
    df_train = df[df['mode'] == 'valid']

    # Extract the relevant features for probabilities (e.g., probText, probAudio, probVideo for both classes)
    feature_columns = ['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1', 
                'probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0', 'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0']

    # Prepare the feature matrix (X) and pseudo-labels (y) for training the Naive Bayes model
    # We'll assume equal prior probabilities for simplicity.
    X = df_train[feature_columns].values
    y = df_train['annotation_label']

    # Train a Gaussian Naive Bayes model
    gnb = GaussianNB()
    gnb.fit(X, y)

    def predict_label(row):
        # Prepare row features for prediction
        row_features = row[feature_columns].values.reshape(1, -1)
        
        # Predict the label using Naive Bayes
        return gnb.predict(row_features)[0]

    # Apply the prediction function to each row of the DataFrame
    df[text] = df.apply(predict_label, axis=1)
    return df

In [44]:
fusion_df = naive_bayes_fusion(fusion_df)

In [45]:
printReport(fusion_df, 'naive_bayes_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.7659    0.6139    0.6815      1023
         1.0     0.7136    0.8367    0.7703      1176

    accuracy                         0.7331      2199
   macro avg     0.7397    0.7253    0.7259      2199
weighted avg     0.7379    0.7331    0.7290      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.6677    0.4076    0.5062       552
         1.0     0.6547    0.8470    0.7385       732

    accuracy                         0.6581      1284
   macro avg     0.6612    0.6273    0.6224      1284
weighted avg     0.6603    0.6581    0.6386      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

# Bagging

## Find oiptimal number of estimators

In [159]:
# # Prepare the features (model probabilities) and the true labels
# X = fusion_df[['probText_0', 'probText_1', 
#                'probAudio_0', 'probAudio_1',
#                'probVideo_0', 'probVideo_1',
#               ]]
# y = fusion_df['annotation_label']

# # Filter out the validation data
# valid_data = fusion_df[fusion_df['mode'] == 'valid']
# X_valid = valid_data[['probText_0', 'probText_1', 
#                        'probAudio_0', 'probAudio_1',
#                        'probVideo_0', 'probVideo_1',
#                       ]]
# y_valid = valid_data['annotation_label']

# # Initialize variables to store results
# best_n_estimators = 0
# best_accuracy = 0

# # Loop over a range of n_estimators (e.g., from 10 to 200)
# for n_estimators in range(1, 201):
#     # Create the base model (Decision Tree) for Bagging
#     base_model = DecisionTreeClassifier()
    
#     # Instantiate the BaggingClassifier with the current n_estimators
#     bagging_model = BaggingClassifier(estimator=base_model, 
#                                       n_estimators=n_estimators, 
#                                       random_state=42)
    
#     # Train the model on the entire training set
#     train_data = fusion_df[fusion_df['mode'] == 'train']
#     X_train = train_data[['probText_0', 'probText_1', 
#                            'probAudio_0', 'probAudio_1',
#                            'probVideo_0', 'probVideo_1',
#                           ]]
#     y_train = train_data['annotation_label']
    
#     bagging_model.fit(X_train, y_train)
    
#     # Predict on the validation set
#     y_pred_valid = bagging_model.predict(X_valid)
    
#     # Calculate the accuracy on the validation set
#     accuracy = accuracy_score(y_valid, y_pred_valid)
    
#     # If the current model has a better accuracy, update the best parameters
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         best_n_estimators = n_estimators

# # Output the best number of estimators and the corresponding accuracy
# print(f"The best number of estimators is: {best_n_estimators}")
# print(f"The corresponding validation set accuracy is: {best_accuracy:.4f}")

## Calculate metrics

In [160]:
# # Prepare the features (model probabilities) and the true labels
# X = fusion_df[['probText_0', 'probText_1', 
#                'probAudio_0', 'probAudio_1',
#                'probVideo_0', 'probVideo_1',
#               ]]
# y = fusion_df['annotation_label']

# # Create the base model (Decision Tree) for Bagging
# base_model = DecisionTreeClassifier()

# # Instantiate BaggingClassifier with Decision Tree as base model
# bagging_model = BaggingClassifier(estimator=base_model, 
#                                   n_estimators=best_n_estimators, 
#                                   random_state=42)

# # Train the model on the training set
# train_data = fusion_df[fusion_df['mode'] == 'train']
# X_train = train_data[['probText_0', 'probText_1', 
#                        'probAudio_0', 'probAudio_1',
#                        'probVideo_0', 'probVideo_1',
#                       ]]
# y_train = train_data['annotation_label']

# bagging_model.fit(X_train, y_train)

# # Predict on the whole dataset
# y_pred = bagging_model.predict(X)

# # Classification report for the entire dataset
# print("Classification Report on the Entire Dataset:")
# print(classification_report(y, y_pred, digits=4))

In [161]:
# for mode in ['train', 'valid', 'test']:
#     subset_data = fusion_df[fusion_df['mode'] == mode]
#     X_subset = subset_data[['probText_0', 'probText_1', 
#                             'probAudio_0', 'probAudio_1',
#                             'probVideo_0', 'probVideo_1',
#                            ]]
#     y_subset = subset_data['annotation_label']
    
#     # Get predictions
#     y_pred = bagging_model.predict(X_subset)
    
#     # Print classification report
#     print(f"\nClassification Report on the {mode.capitalize()} Subset:")
#     print(classification_report(y_subset, y_pred, digits=4))

# Fully Connected Layer

https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def train_mlp_using_labels(df, train_mode='train'):
    """
    Train an MLP classifier using labels as features on the 'train' subset, 
    perform grid search to maximize accuracy on the 'valid' subset, and 
    predict final labels for all rows.

    Args:
        df (pd.DataFrame): Input DataFrame with columns:
            'textLabel', 'audioLabel', 'videoLabel', 'annotation_label', 'mode'.
        train_mode (str): The mode to filter the training subset (default: 'train').
    Returns:
        pd.DataFrame: DataFrame with an additional column 'mlp_label' containing the predicted labels.
    """
    # Filter the training and validation subsets
    train_df = df[df['mode'] == train_mode]
    # train_df = df[df['mode'].isin(['train', 'valid'])]
    # valid_df = df[df['mode'] == valid_mode]

    # Features (labels from text, audio, and video)
    X_train = train_df[['textLabel', 'audioLabel', 'videoLabel', 'trimodal_preds', 'text_audio_preds', 'audio_video_preds', 'text_video_preds']]
    y_train = train_df['annotation_label']
    # X_valid = valid_df[['textLabel', 'audioLabel', 'videoLabel']]
    # y_valid = valid_df['annotation_label']

    # Define hyperparameter grid for grid search
    param_grid = {
        'hidden_layer_sizes': [(64,32), (32,16), (32,), (64,), (128,), (256,), (128, 64), (128, 32), (256, 128), (256, 32), (128, 64, 32), (256, 128, 64), (256, 128, 32), (256, 128, 64, 32), ],
        'activation': ['relu', 'tanh'],
        'learning_rate_init': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3],
        'batch_size': [8, 16, 32, 64, 128],
        'max_iter': [500]
    }

    # Initialize the MLPClassifier
    mlp = MLPClassifier(early_stopping=True, tol=1e-5)

    # Perform grid search with accuracy as the scoring metric
    grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Select the best estimator based on grid search
    best_mlp = grid_search.best_estimator_
    print("Best params: ", grid_search.best_params_)
    print("Num of iterations: ", best_mlp.n_iter_)

    # # Evaluate the best model on the validation subset
    # valid_predictions = best_mlp.predict(X_valid)
    
    # valid_accuracy = accuracy_score(y_valid, valid_predictions)
    # print(f"Validation Accuracy of Best Model: {valid_accuracy:.4f}")

    # Use the trained best MLP model to predict for the entire dataset
    all_features = df[['textLabel', 'audioLabel', 'videoLabel', 'trimodal_preds', 'text_audio_preds', 'audio_video_preds', 'text_video_preds']]
    df['mlp_label'] = best_mlp.predict(all_features)

    return df

In [10]:
fusion_df = train_mlp_using_labels(fusion_df, train_mode='train')

/home/k/kyparkypar/ondemand/data/sys/myjobs/projects/default/my_enviroment/lib64/python3.9/site-packages/sklearn/model_selection/_search.py:412: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


Best params:  {'activation': 'relu', 'batch_size': 16, 'hidden_layer_sizes': (32, 16), 'learning_rate_init': 0.0005, 'max_iter': 500}
Num of iterations:  14


In [11]:
printReport(fusion_df, 'mlp_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8714    0.8876    0.8794      1023
         1.0     0.9006    0.8861    0.8933      1176

    accuracy                         0.8868      2199
   macro avg     0.8860    0.8868    0.8863      2199
weighted avg     0.8870    0.8868    0.8868      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8914    0.9221    0.9065       552
         1.0     0.9397    0.9153    0.9273       732

    accuracy                         0.9182      1284
   macro avg     0.9156    0.9187    0.9169      1284
weighted avg     0.9189    0.9182    0.9184      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8144    0.8587    0.8360        92
         1.0     0.9015    0.8686    0.8848       137

    accuracy                         0.8646       229
 

In [12]:
# from sklearn.neural_network import MLPClassifier
# import pandas as pd

# def train_mlp_using_probabilities(df, mode='valid'):
#     """
#     Train an MLP classifier using probabilities as features from 'text', 'audio', and 'video'
#     on the 'valid' subset and predict the final labels for all rows.

#     Args:
#         df (pd.DataFrame): Input DataFrame with columns:
#             'probText_0', 'probText_1', 'probAudio_0', 'probAudio_1',
#             'probVideo_0', 'probVideo_1', 'annotation_label', 'mode'.
#         mode (str): The mode to filter the validation subset (default: 'valid').

#     Returns:
#         pd.DataFrame: DataFrame with an additional column 'mlp_label' containing the predicted labels.
#     """
#     # Filter the validation subset
#     valid_df = df[df['mode'] == mode]
    
#     # Features (probabilities from text, audio, and video)
#     X = valid_df[['probText_0', 'probText_1', 'probAudio_0', 'probAudio_1', 'probVideo_0', 'probVideo_1']]
    
#     # Target (ground truth labels)
#     y = valid_df['annotation_label']
    
#     # Train the MLP classifier
#     mlp = MLPClassifier(hidden_layer_sizes=(64, ), max_iter=1000000000)
#     mlp.fit(X, y)
    
#     # Use the trained MLP model to predict for the entire dataset
#     all_features = df[['probText_0', 'probText_1', 'probAudio_0', 'probAudio_1', 'probVideo_0', 'probVideo_1']]
#     df['mlpProb_label'] = mlp.predict(all_features).astype('int')
    
#     return df

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def train_mlp_using_probabilities(df, train_mode='train'):
    """
    Train an MLP classifier using probabiliy scores as features on the 'train' subset, 
    perform grid search to maximize accuracy on the 'valid' subset, and 
    predict final labels for all rows.
    """
    # Filter the training and validation subsets
    train_df = df[df['mode'] == train_mode]
    # train_df = df[df['mode'].isin(['train', 'valid'])]
    # valid_df = df[df['mode'] == valid_mode]

    # Features (labels from text, audio, and video)
    X_train = train_df[['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1', 
                'probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0', 'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0']]
    y_train = train_df['annotation_label']
    # X_valid = valid_df[['probText_0', 'probText_1', 'probAudio_0', 'probAudio_1', 'probVideo_0', 'probVideo_1']]
    # y_valid = valid_df['annotation_label']

    # Define hyperparameter grid for grid search
    param_grid = {
        'hidden_layer_sizes': [(64,32), (32,16), (32,), (64,), (128,), (256,), (128, 64), (128, 32), (256, 128), (256, 32), (128, 64, 32), (256, 128, 64), (256, 128, 32), (256, 128, 64, 32), ],
        'activation': ['relu', 'tanh'],
        'learning_rate_init': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3],
        'batch_size': [8, 16, 32, 64, 128],
        'max_iter': [500]
    }

    # Initialize the MLPClassifier
    mlp = MLPClassifier(early_stopping=True, tol=1e-5)

    # Perform grid search with accuracy as the scoring metric
    grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Select the best estimator based on grid search
    best_mlp = grid_search.best_estimator_
    print("Best params: ", grid_search.best_params_)
    print("Num of iterations: ", best_mlp.n_iter_)

    # # Evaluate the best model on the validation subset
    # valid_predictions = best_mlp.predict(X_valid)
    
    # valid_accuracy = accuracy_score(y_valid, valid_predictions)
    # print(f"Validation Accuracy of Best Model: {valid_accuracy:.4f}")

    # Use the trained best MLP model to predict for the entire dataset
    all_features = df[['probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1', 'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1', 
                'probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0', 'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0']]
    df['mlpProb_label'] = best_mlp.predict(all_features)

    return df

In [14]:
fusion_df = train_mlp_using_probabilities(fusion_df, train_mode='train')

/home/k/kyparkypar/ondemand/data/sys/myjobs/projects/default/my_enviroment/lib64/python3.9/site-packages/sklearn/model_selection/_search.py:412: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


Best params:  {'activation': 'relu', 'batch_size': 64, 'hidden_layer_sizes': (256, 128, 64), 'learning_rate_init': 0.001, 'max_iter': 500}
Num of iterations:  15


In [15]:
printReport(fusion_df, 'mlpProb_label')

Classification Report for the Whole Dataset:
              precision    recall  f1-score   support

         0.0     0.8632    0.9130    0.8874      1023
         1.0     0.9203    0.8741    0.8966      1176

    accuracy                         0.8922      2199
   macro avg     0.8918    0.8936    0.8920      2199
weighted avg     0.8938    0.8922    0.8923      2199


Classification Report for Train Subset:
              precision    recall  f1-score   support

         0.0     0.8861    0.9583    0.9208       552
         1.0     0.9665    0.9071    0.9359       732

    accuracy                         0.9291      1284
   macro avg     0.9263    0.9327    0.9283      1284
weighted avg     0.9319    0.9291    0.9294      1284


Classification Report for Valid Subset:
              precision    recall  f1-score   support

         0.0     0.8081    0.8696    0.8377        92
         1.0     0.9077    0.8613    0.8839       137

    accuracy                         0.8646       229
 

---
---

In [219]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.preprocessing import StandardScaler

# class MultiHeadAttentionClassifier(nn.Module):
#     def __init__(self, input_dim, num_heads=7, hidden_dim=64, output_dim=2):
#         super(MultiHeadAttentionClassifier, self).__init__()
#         self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.Tanh(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.Tanh(),
#             nn.Linear(hidden_dim, output_dim)
#         )
    
#     def forward(self, x):
#         attn_output, _ = self.attention(x, x, x)
#         out = self.fc(attn_output[:, 0, :])  # Take first token output
#         return out

In [220]:
# def train_multihead_attention(df, epochs=3, batch_size=64, lr=0.0001, text='mha_label'):
#     features = df[['probText_0', 'probAudio_0', 'probVideo_0', 'trimodal_prob_0',
#                    'text_audio_prob_0', 'audio_video_prob_0', 'text_video_prob_0',
#                    'probText_1', 'probAudio_1', 'probVideo_1', 'trimodal_prob_1',
#                    'text_audio_prob_1', 'audio_video_prob_1', 'text_video_prob_1']].values
#     labels = df['annotation_label'].values
    
#     # Normalize the features
#     scaler = StandardScaler()
#     features = scaler.fit_transform(features)
    
#     # Convert to tensors
#     X = torch.tensor(features, dtype=torch.float32)
#     y = torch.tensor(labels, dtype=torch.long)
    
#     data_loaders = {}
#     for mode in ['train', 'valid', 'test']:
#         mask = df['mode'] == mode
#         if mask.sum() > 0:  # Ensure mode has data
#             X_subset = X[mask].unsqueeze(1)  # Ensure correct dimensions
#             if mode != 'test':
#                 y_subset = y[mask]
#                 dataset = TensorDataset(X_subset, y_subset)
#             else:
#                 dataset = TensorDataset(X_subset)  # No labels for test mode
#             shuffle = mode == 'train'
#             data_loaders[mode] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    
#     model = MultiHeadAttentionClassifier(input_dim=14)  # 14 input probabilities
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
#     # Training loop
#     for epoch in range(epochs):
#         model.train()
#         for inputs, targets in data_loaders['train']:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             loss.backward()
#             optimizer.step()
    
#     # Evaluation for all modes
#     model.eval()
#     for mode in ['train', 'valid', 'test']:
#         if mode in data_loaders:
#             predictions = []
#             with torch.no_grad():
#                 for batch in data_loaders[mode]:
#                     inputs = batch[0]
#                     outputs = model(inputs)
#                     preds = torch.argmax(outputs, dim=1)
#                     predictions.extend(preds.numpy())
#             df.loc[df['mode'] == mode, text] = predictions  # Assign predictions back to df
    
#     return df

In [221]:
# temp_df = train_multihead_attention(fusion_df, epochs=30, batch_size=128, lr=0.0005, text='mha_label')

In [222]:
# temp_df

In [223]:
# printReport(temp_df, 'mha_label')